# Preliminaries

## Libraries

In [1]:
import os
import logging
import requests
import pandas as pd
import yaml

<br>
<br>

## Logging

In [2]:
logging.basicConfig(level=logging.INFO,
                    format='\n\n%(message)s\n%(asctime)s.%(msecs)03d',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

<br>
<br>

## Set-up

Paths

In [3]:
os.path.join(os.getcwd())

'/content'

<br>

Keys

In [4]:
pathstr = os.path.join(os.getcwd(), 'services.yaml')

In [5]:
with open(file=pathstr, mode='r') as stream:
    try:
        nodes = yaml.load(stream=stream, Loader=yaml.CLoader)
    except yaml.YAMLError as err:
        raise Exception(err)

In [6]:
dictionary: dict = [item for item in nodes['services'] if item['code'] == 'thames'][0]

<br>

Hence

In [8]:
root = 'https://prod-tw-opendata-app.uk-e1.cloudhub.io'

In [9]:
alerts = '/data/STE/v1/DischargeAlerts'
states = '/data/STE/v1/DischargeCurrentStatus'

In [10]:
params={'limit': 1000, 'offset': 0}

In [11]:
url = '{root}{resource}'

<br>
<br>

# Explore

##  A Digression

### Alerts

Query

In [134]:
params_={'limit': 1000, 'offset': 0, 'col_1': 'DateTime', 'operand_1': 'gte', 'value_1': '2022-03-01',
         'col_2': 'DateTime', 'operand_2': 'lt', 'value_2': '2022-04-01'}

r = requests.get(url=url.format(root=root, resource=alerts),
                 headers={'client_id': dictionary['client_id'],'client_secret': dictionary['client_secret']},
                 params=params_)

r.url

<br>

Unload

In [136]:
if r.status_code == 200:
    response = r.json()
else:
    raise Exception(f'{r.status_code}\n{r.json()}')

if 'items' in response:
    data = pd.json_normalize(response, 'items')
else:
    data = pd.DataFrame()

data.info()

### States

Query

In [151]:
params_={'limit': 1000, 'offset': 0, 'col_1': 'StatusChange', 'operand_1': 'gte', 'value_1': '2022-04-01',
         'col_2': 'StatusChange', 'operand_2': 'lt', 'value_2': '2022-05-01'}

r = requests.get(url=url.format(root=root, resource=states),
                 headers={'client_id': dictionary['client_id'],'client_secret': dictionary['client_secret']},
                 params=params_)

r.url

'https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeCurrentStatus?limit=1000&offset=0&col_1=StatusChange&operand_1=gte&value_1=2022-04-01&col_2=StatusChange&operand_2=lt&value_2=2022-05-01'

Unload

In [152]:
if r.status_code == 200:
    response = r.json()
else:
    raise Exception(f'{r.status_code}\n{r.json()}')

if 'items' in response:
    data = pd.json_normalize(response, 'items')
else:
    data = pd.DataFrame()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   LocationName                   38 non-null     object
 1   PermitNumber                   38 non-null     object
 2   LocationGridRef                38 non-null     object
 3   X                              38 non-null     int64 
 4   Y                              38 non-null     int64 
 5   ReceivingWaterCourse           38 non-null     object
 6   AlertStatus                    38 non-null     object
 7   StatusChange                   38 non-null     object
 8   AlertPast48Hours               38 non-null     bool  
 9   MostRecentDischargeAlertStart  2 non-null      object
 10  MostRecentDischargeAlertStop   2 non-null      object
dtypes: bool(1), int64(2), object(8)
memory usage: 3.1+ KB


<br>
<br>

## Alerts

Query

In [73]:
r = requests.get(url=url.format(root=root, resource=alerts),
                 headers={'client_id': dictionary['client_id'],'client_secret': dictionary['client_secret']},
                 params=params)

In [74]:
r.url

'https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeAlerts?limit=1000&offset=0'

<br>

Unload

In [75]:
if r.status_code == 200:
    response = r.json()
    data = pd.json_normalize(response, 'items')
else:
    raise Exception(f'{r.status_code}\n{r.json()}')

In [76]:
data.head()

,LocationName,PermitNumber,LocationGridRef,X,Y,ReceivingWaterCourse,AlertType,DateTime
0,"Low Level 1 N, Grosvenor Ditch, Millb",TEMP.2752,TQ30277879,530270,178790,River Thames,Stop,2023-06-29T05:26:00
1,Pauls Pier,TEMP.2968,TQ32008081,532000,180810,River Thames,Stop,2023-06-29T05:22:00
2,North Weald,TEMP.2809,TL49570472,549570,204720,North Weald Brook,Start,2023-06-29T05:15:00
3,Frogmore Storm,TEMP.2599,TQ254749,525400,174900,Bell Lane Creek,Stop,2023-06-29T05:10:00
4,"Leopold Road, Church Path",TEMP.2737,TQ21048487,521040,184870,Mitchell Brook,Stop,2023-06-29T05:00:00


In [79]:
if 'items' in response:
    print(len(response['items']))

1000


<br>
<br>

## States

Query

In [16]:
r = requests.get(url=url.format(root=root, resource=states),
                 headers={'client_id': dictionary['client_id'],'client_secret': dictionary['client_secret']},
                 params=params)

In [17]:
r.url

'https://prod-tw-opendata-app.uk-e1.cloudhub.io/data/STE/v1/DischargeCurrentStatus?limit=1000&offset=0'

<br>

Unload

In [18]:
if r.status_code == 200:
    response = r.json()
    data = pd.json_normalize(response, 'items')
else:
    raise Exception(f'{r.status_code}\n{r.json()}')

In [19]:
data.head()

,LocationName,PermitNumber,LocationGridRef,X,Y,ReceivingWaterCourse,AlertStatus,StatusChange,AlertPast48Hours,MostRecentDischargeAlertStart,MostRecentDischargeAlertStop
0,(Northern) Low Level No 1 Brook Green,CSAB.0535,TQ23047815,523040,178150,River Thames,Not discharging,2022-09-13T16:18:00,False,2022-09-13T16:08:00,2022-09-13T16:18:00
1,"15 Coldharbour Lane, Bushey",TEMP.2331,TQ13909538,513900,195380,Colne Brook,Not discharging,2022-04-01T00:00:00,False,None,None
2,Abbess Roding,CTCR.2034,TL57581109,557580,211090,Coopers Brook,Not discharging,2023-05-21T10:00:00,False,2023-05-21T09:30:00,2023-05-21T10:00:00
3,Abbey Mills,CSAB.0530,TQ38858314,538850,183140,River Lee (Channelsea River South),Not discharging,2022-09-01T20:45:00,False,None,None
4,Abingdon,CTCR.1804,SU49339509,449330,195090,Odhay Hill Ditch,Not discharging,2023-05-09T00:30:00,False,2023-05-08T23:30:00,2023-05-09T00:30:00


<br>

## Options


```bash
    curl -G "https://environment.data.gov.uk/public-register/api/search.json" --data-urlencode name-search=*thames
```

```bash
    curl -G "https://environment.data.gov.uk/public-register/api/completion.json" --data-urlencode "number=TH/"
```